In [114]:
import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn import set_config

In [115]:
url = 'https://raw.githubusercontent.com/Jacoposigno1999/interviews-data-science-assignment/main/datasets/diamonds/diamonds.csv'
df = pd.read_csv(url)
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.10,Ideal,H,SI2,62.0,55.0,4733,6.61,6.65,4.11
1,1.29,Ideal,H,SI1,62.6,56.0,6424,6.96,6.93,4.35
2,1.20,Premium,I,SI1,61.1,58.0,5510,6.88,6.80,4.18
3,1.50,Ideal,F,SI1,60.9,56.0,8770,7.43,7.36,4.50
4,0.90,Very Good,F,VS2,61.7,57.0,4493,6.17,6.21,3.82
...,...,...,...,...,...,...,...,...,...,...
4995,0.38,Ideal,H,VVS2,62.3,53.3,832,4.65,4.69,2.91
4996,0.33,Premium,G,VVS2,61.3,59.0,927,4.45,4.42,2.72
4997,1.25,Ideal,J,VVS2,62.1,56.0,5980,6.81,6.84,4.24
4998,0.31,Premium,F,VS2,62.9,58.0,802,4.31,4.27,2.70


In [116]:
from sklearn.base import BaseEstimator, TransformerMixin

In [117]:
class FeatureDropper(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y = None):
        return self
    
    def trasformer(self, X):
        return X.drop(["y","z"], axis = 1)

## Pipeline for numerical features


In [118]:
numeric_processor=Pipeline(
    steps=[("imputation_mean",SimpleImputer(missing_values=np.nan,strategy="mean")),
           ("featuredropper", FeatureDropper()),
           ("scaler",MinMaxScaler())]

)

numeric_processor

Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('featuredropper', FeatureDropper()),
                ('scaler', MinMaxScaler())])

## Pipeline for categorical data 

In [119]:
from sklearn.preprocessing import OneHotEncoder

categorical_processor=Pipeline(
    steps=[("imputation_consatnt",SimpleImputer(fill_value="missing",strategy="constant")),
           ("featuredropper", FeatureDropper()),
           ("onehot",OneHotEncoder(handle_unknown="ignore"))]

)

categorical_processor

Pipeline(steps=[('imputation_consatnt',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('featuredropper', FeatureDropper()),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

## Final pipeline

In [120]:
X[:1]

,carat,cut,color,clarity,depth,table,x,y,z
0,1.1,Ideal,H,SI2,62.0,55.0,6.61,6.65,4.11


In [121]:
## combine processing technqiues
from sklearn.compose import ColumnTransformer

In [122]:
Variabili_numeriche = ["carat", "depth", "table", "x", "y", "z"]
#df.select_dtypes(exclude = 'object').columns

In [123]:
Variabili_categoriche =  ["cut", "color", "clarity"]
#df.select_dtypes('object').columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [124]:
preprocessor=ColumnTransformer(
    [("categorical",categorical_processor, Variabili_categoriche),
    ("numerical",numeric_processor,Variabili_numeriche)]

)

In [125]:
from sklearn.pipeline import make_pipeline

In [126]:
pipe=make_pipeline(preprocessor,linear_model.LinearRegression())
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_consatnt',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('featuredropper',
                                                                   FeatureDropper()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['cut', 'color', 'clarity']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('featuredropper',
                                                                   FeatureDropper()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['carat', 'depth', 'table',
                                                   'x', 'y', 'z'])])),
                ('linearregression', LinearRegression())])

### Train the model 


In [127]:
#Removing negative prices
id_0 = df[df['price']<0].index
df = df.drop(id_0)

y = np.log(df['price'])  #take a log trasformation of the response variable to try to avoid heteroschedasticity
X = df.drop('price', axis=1)

In [128]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.10,Ideal,H,SI2,62.0,55.0,6.61,6.65,4.11
1,1.29,Ideal,H,SI1,62.6,56.0,6.96,6.93,4.35
2,1.20,Premium,I,SI1,61.1,58.0,6.88,6.80,4.18
3,1.50,Ideal,F,SI1,60.9,56.0,7.43,7.36,4.50
4,0.90,Very Good,F,VS2,61.7,57.0,6.17,6.21,3.82
...,...,...,...,...,...,...,...,...,...
4995,0.38,Ideal,H,VVS2,62.3,53.3,4.65,4.69,2.91
4996,0.33,Premium,G,VVS2,61.3,59.0,4.45,4.42,2.72
4997,1.25,Ideal,J,VVS2,62.1,56.0,6.81,6.84,4.24
4998,0.31,Premium,F,VS2,62.9,58.0,4.31,4.27,2.70


In [129]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 0)



In [130]:
pipe.fit(X_train, y_train)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'FeatureDropper()' (type <class '__main__.FeatureDropper'>) doesn't

In [113]:
pipe.score(X_train, y_train)

0.9576521435572023

In [15]:
pipe.score(X_test, y_test)

0.9725429311298751